In [3]:
# ------- SERVER EXTENSIONS ---------
lib =  r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities/droplet_dataset'
lib2 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities'
lib3 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/data_analysis'
lib4 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy'
lib5 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/scripts'
import sys
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)
from utilities.general_helpers import *
# ------- SERVER EXTENSIONS ---------
import numpy as np
import pandas as pd
import scipy
import sklearn
from sklearn.manifold import TSNE
import pickle
# from Bio.Cluster import kcluster
import os
import numpy as np
import yaml
from os.path import join
import os
import pandas
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sys
import seaborn as sns
# import statsmodels as sm
import scipy.stats as stats
from scipy.stats import rankdata
from Bio.Cluster import kcluster
from sklearn.manifold import TSNE
import pickle
import numpy as np
from utilities.general_helpers import flatten_list

# import pyclustering
from shutil import copyfile
import matplotlib as plt


In [12]:
def multipletests_fdr(p_vals, alpha=0.05):
    """
    Test results and p-value correction for multiple tests using Benjamini/Hochberg method.
    :param p_vals: np.array of pvals
    :return: pvals_corrected
    """

    ranked_p_values = rankdata(p_vals)
    pvals_corrected = p_vals * len(p_vals) / ranked_p_values
    pvals_corrected[pvals_corrected > 1] = 1

    return pvals_corrected < alpha, pvals_corrected

In [4]:
OUTPUT_PATH = r'/storage/md_keren/shitay/outputs/clustering/cluster_analysis/cluster_analysis_24.5.21'
FILTERED_CELLS_PATH = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/5.21/cohort_normalized_24.5.21.pkl'
KMEANS_ROW_CLUSTERS_PATH = r'/storage/md_keren/shitay/outputs/clustering/kmeans/24.5.21/row_kmeans'
KMEANS_FILE_NAME = r'kmeans_immune_cells_4k_genes'  # excluding the suffix: '_k_num.pkl'


In [6]:
K = 4
print(f'Loading cohort file:\n{FILTERED_CELLS_PATH}')
filtered_cells = pickle.load(open(FILTERED_CELLS_PATH, 'rb'))
filtered_cells = filtered_cells.filter_cells_by_property('is_immune', True)

kmeans_file_path = join(KMEANS_ROW_CLUSTERS_PATH, KMEANS_FILE_NAME + f'_k_{K}.pkl')
print(f'Loading kmeans file:\n{kmeans_file_path}')
clusters = pickle.load(open(kmeans_file_path, 'rb'))['clusters']

# markers = find_markers_in_clusters(filtered_cells, clusters)

In [100]:
data_rna_seq = filtered_cells
clusters_indices = clusters
log_ratio_threshold = 0.25
pval_threshold=0.05

In [8]:
cluster_markers_list = []
cluster_idx = 2
current_cluster_indices = clusters_indices[cluster_idx]
other_clusters_indices = [ii for ii in flatten_list(clusters_indices) if not ii in clusters_indices[cluster_idx]]

# cluster_markers = find_marker_genes_in_cluster(data_rna_seq[current_cluster_indices], data_rna_seq[other_clusters_indices], log_ratio_threshold, pval_threshold)
# cluster_markers_list.append({'cluster id': cluster_idx, 'markers': cluster_markers})
# return cluster_markers_list

In [9]:
cluster_data = data_rna_seq[current_cluster_indices] 
other_clusters_data = data_rna_seq[other_clusters_indices]
log_FC_threshold = log_ratio_threshold
pval_threshold = pval_threshold

# Debug zone

In [84]:
# FC = np.log2((np.mean(cluster_data.counts, axis=0) + 0.01) / (np.mean(other_clusters_data.counts, axis=0) + 0.01))
df = pd.DataFrame()

In [85]:
"""
min_diff_pct:
1. minimum percent difference between the percent of cells expressing the gene in the cluster
and the percent of cells expressing gene in all other clusters combined.

min_pct: 0.1
2. only test genes that are detected in a minimum fraction of cells in either of the two populations.
"""

min_expression_threshold = 1

df['features'] = cluster_data.features
df['gene names'] = cluster_data.gene_names
df['(1)mean_expression'] = np.mean(cluster_data.counts, axis=0)
df['(2)mean expression'] = np.mean(other_clusters_data.counts, axis=0)
df['log_FC'] = np.log2((df['(1)mean_expression'] + 0.01) / (df['(2)mean expression'] + 0.01))
df['(1)#expressing'] = np.sum((cluster_data.counts > min_expression_threshold), axis=0)
df['(2)#expressing'] = np.sum((other_clusters_data.counts > min_expression_threshold), axis=0)
df['(1)%expressing'] = df['(1)#expressing'] / n_cells_in_cluster
df['(2)%expressing'] = df['(2)#expressing'] / n_cells_in_other_clusters
df['%expressing_diff'] = df['(1)%expressing'] - df['(2)%expressing']

In [108]:
min_pct = 0.1
min_diff_pct = 0.1
df2 = df[(df["(1)%expressing"] > min_pct)]
df2 = df2[(df2["%expressing_diff"] > min_diff_pct)]
df2 = df2[(df2['log_FC'] > log_ratio_threshold)]
df2 = df2.sort_values(['log_FC'], ascending=False)
df2.head()

,gene names,(1)mean_expression,(2)mean expression,log_FC,(1)#expressing,(2)#expressing,(1)%expressing,(2)%expressing,%expressing_diff
5965,VPREB3,0.359311,0.001394,5.018502,3914,31,0.124238,0.000594,0.123644
14146,CD19,0.308011,0.001271,4.818354,3686,29,0.117001,0.000556,0.116445
9988,MS4A1,0.709505,0.023614,4.419866,5975,446,0.189658,0.008551,0.181107
9584,BANK1,0.275032,0.005316,4.218011,3382,132,0.107351,0.002531,0.104821
3078,CD79A,0.670265,0.036945,3.857042,6157,768,0.195436,0.014725,0.180711


In [129]:
n_cells_in_cluster = len(cluster_data.counts)
n_cells_in_other_clusters = len(other_clusters_data.counts)
p_values = []
for index, row in df2.iterrows():
    number_of_cells_expressing_gene_in_cluster = row["(1)#expressing"]
    number_of_cells_expressing_gene_in_other_clusters = row["(2)#expressing"]

    oddsratio, pvalue = stats.fisher_exact([[number_of_cells_expressing_gene_in_cluster,
                                             number_of_cells_expressing_gene_in_other_clusters],
                                            [n_cells_in_cluster - number_of_cells_expressing_gene_in_cluster,
                                             n_cells_in_other_clusters - number_of_cells_expressing_gene_in_other_clusters]])

    p_values.append(pvalue)

reject_arr, pvals_corrected = multipletests_fdr(np.array(p_values), alpha=0.05)

In [131]:
# pvals_corrected
df2['pvals'] = pvals_corrected.tolist()

df2.shape
# pvals_corrected.shape
# len(p_values)
count=1
for index, row in df2.iterrows():
    count+=1
df2

,gene names,(1)mean_expression,(2)mean expression,log_FC,(1)#expressing,(2)#expressing,(1)%expressing,(2)%expressing,%expressing_diff,pvals
5965,VPREB3,0.359311,0.001394,5.018502,3914,31,0.124238,0.000594,0.123644,0.000000e+00
14146,CD19,0.308011,0.001271,4.818354,3686,29,0.117001,0.000556,0.116445,0.000000e+00
9988,MS4A1,0.709505,0.023614,4.419866,5975,446,0.189658,0.008551,0.181107,0.000000e+00
9584,BANK1,0.275032,0.005316,4.218011,3382,132,0.107351,0.002531,0.104821,0.000000e+00
3078,CD79A,0.670265,0.036945,3.857042,6157,768,0.195436,0.014725,0.180711,0.000000e+00
5791,CCR7,0.700436,0.061396,3.314793,8905,1274,0.282663,0.024426,0.258236,0.000000e+00
7717,LEF1,0.558654,0.067047,2.883736,7335,1521,0.232828,0.029162,0.203666,0.000000e+00
13124,MAL,0.556834,0.077763,2.691235,7080,1628,0.224733,0.031213,0.193520,0.000000e+00
29802,CTB-133G6.1,0.323772,0.052310,2.421336,4688,1204,0.148807,0.023084,0.125722,0.000000e+00
1523,TCF7,0.532108,0.091339,2.419397,6939,1928,0.220258,0.036965,0.183292,0.000000e+00


In [111]:
filtered_genes_indices = df2.index.tolist()
filtered_genes_indices

for gene_idx in filtered_genes_indices:
    cluster_gene_expression = cluster_data.counts[:, gene_idx] > 1
    other_clusters_gene_expression = other_clusters_data.counts[:, gene_idx] > 1

    number_of_cells_expressing_gene_in_cluster = sum(cluster_gene_expression)
    number_of_cells_expressing_gene_in_other_clusters = sum(other_clusters_gene_expression)

    n_cells_in_cluster = len(cluster_gene_expression)
    n_cells_in_other_clusters = len(other_clusters_gene_expression)

    oddsratio, pvalue = stats.fisher_exact([[number_of_cells_expressing_gene_in_cluster,
                                             number_of_cells_expressing_gene_in_other_clusters],
                                            [n_cells_in_cluster - number_of_cells_expressing_gene_in_cluster,
                                             n_cells_in_other_clusters - number_of_cells_expressing_gene_in_other_clusters]])

    percentage_voting_expression_in_clusters.append([[number_of_cells_expressing_gene_in_cluster, number_of_cells_expressing_gene_in_other_clusters],
                                                     [n_cells_in_cluster, n_cells_in_other_clusters]])
    p_values.append(pvalue)

reject_arr, pvals_corrected = multipletests_fdr(np.array(p_values), alpha=0.05)

In [113]:
len(filtered_genes_indices)

# df
# df['cluster num cells expressing'] / 

109

In [99]:
df.sort_values(['log_FC'], ascending=False).reset_index(drop=True)
# df.sort_values(['(1)%expressing'], ascending=False).reset_index(drop=True).head(40)
# df.sort_values(['%expressing_diff'], ascending=False).reset_index(drop=True).head(40)

,gene names,(1)mean_expression,(2)mean expression,log_FC,(1)#expressing,(2)#expressing,(1)%expressing,(2)%expressing,%expressing_diff
0,VPREB3,0.359311,0.001394,5.018502,3914,31,0.124238,0.000594,0.123644
1,CD19,0.308011,0.001271,4.818354,3686,29,0.117001,0.000556,0.116445
2,TCL1A,0.316209,0.003551,4.589352,2650,55,0.084116,0.001055,0.083062
3,MS4A1,0.709505,0.023614,4.419866,5975,446,0.189658,0.008551,0.181107
4,LINC00926,0.264060,0.004576,4.232798,3110,113,0.098718,0.002167,0.096551
5,BANK1,0.275032,0.005316,4.218011,3382,132,0.107351,0.002531,0.104821
6,CD79A,0.670265,0.036945,3.857042,6157,768,0.195436,0.014725,0.180711
7,RP5-887A10.1,0.130879,0.000668,3.723048,1512,17,0.047994,0.000326,0.047668
8,RP11-693J15.5,0.149605,0.002208,3.708575,1959,41,0.062183,0.000786,0.061396
9,FCRLA,0.159795,0.002999,3.707314,2132,68,0.067674,0.001304,0.066370


In [ ]:
df['log_FC'] = np.log2((np.mean(cluster_data.counts, axis=0) + 0.01) / (np.mean(other_clusters_data.counts, axis=0) + 0.01))


df['n_expressing_cells__cls1 > n_expressing_cells__cls2'] = percentage_voting_expression_in_clusters[:,0] > percentage_voting_expression_in_clusters[:, 1]
df['features'] = cluster_data.features
df['gene names'] = cluster_data.gene_names





--------------------------------------

In [ ]:
# def find_marker_genes_in_cluster(cluster_data, other_clusters_data, log_FC_threshold, pval_threshold):
"""
After the clustering process has been done run this function to find marker genes for each cluster.
The function conducts a Fisher Exact Test for every gene to check whether that gene constitutes a marker
of one of the clusters. Obviously, we conduct a test per cluster. that means that if we have k clusters and n
genes we will have k*n tests.
We test whether the gene in the cluster expresses differently of the other clusters.
we define expressed as value > 1 (after normalization). and we test the proportion between the number of cells
expressing each gene compered to all other clusters together.
because we conduct many statistical test we will correct the pvalues using Benjamini/Hochberg correction
:param Data_RNAseq: Can be cohort object or singular RNAseq object which all its cells having association to one of the clusters.
:param clusters_list: cluster in size of number of cells such that each place indicate which cluster corresponds
to that cell (number of cluster).

Note: Marker defined as gene with pval < pval_thresh and log_FC < log_ratio_threshold
:return:
gene names - of all gene markers, ordered by pval_corrected
gene ids - of all gene markers, ordered by pval_corrected
pval - of all gene markers, ordered by pval_corrected
log ratios - of all gene markers, ordered by pval_corrected
"""

# Part 1
percentage_voting_expression_in_clusters = []
p_values = []  # before correction

# statistical test for each gene
for gene_idx in range(cluster_data.number_of_genes):
    cluster_gene_expression = cluster_data.counts[:, gene_idx] > 1
    other_clusters_gene_expression = other_clusters_data.counts[:, gene_idx] > 1

    number_of_cells_expressing_gene_in_cluster = sum(cluster_gene_expression)
    number_of_cells_expressing_gene_in_other_clusters = sum(other_clusters_gene_expression)

    n_cells_in_cluster = len(cluster_gene_expression)
    n_cells_in_other_clusters = len(other_clusters_gene_expression)

    oddsratio, pvalue = stats.fisher_exact([[number_of_cells_expressing_gene_in_cluster,
                                             number_of_cells_expressing_gene_in_other_clusters],
                                            [n_cells_in_cluster - number_of_cells_expressing_gene_in_cluster,
                                             n_cells_in_other_clusters - number_of_cells_expressing_gene_in_other_clusters]])

    percentage_voting_expression_in_clusters.append([[number_of_cells_expressing_gene_in_cluster, number_of_cells_expressing_gene_in_other_clusters],
                                                     [n_cells_in_cluster, n_cells_in_other_clusters]])
    p_values.append(pvalue)

reject_arr, pvals_corrected = multipletests_fdr(np.array(p_values), alpha=0.05)

# Part 2
percentage_voting_expression_in_clusters = np.array(percentage_voting_expression_in_clusters)
percentage_voting_expression_in_clusters = percentage_voting_expression_in_clusters[:,
                                           0] / percentage_voting_expression_in_clusters[:, 1]

df = pd.DataFrame(np.array([p_values, pvals_corrected]).T, columns=['pval', 'corected_pval'])
df['log_FC'] = np.log2((np.mean(cluster_data.counts, axis=0) + 0.01) / (np.mean(other_clusters_data.counts, axis=0) + 0.01))
df['cluster mean_expression'] = np.mean(cluster_data.counts, axis=0)
df['other clusters mean expression'] = np.mean(other_clusters_data.counts, axis=0)
df['n_expressing_cells__cls1 > n_expressing_cells__cls2'] = percentage_voting_expression_in_clusters[:,0] > percentage_voting_expression_in_clusters[:, 1]
df['features'] = cluster_data.features
df['gene names'] = cluster_data.gene_names

# sort log_FC by size, and adjust features/genes and percentage_voting_expression_in_clusters by the log_FC.
df = df.sort_values(['log_FC'], ascending=False)

# take only genes with pval < pval_threshold
df = df[df['corected_pval'] < pval_threshold]
# take only genes with log_FC > log_FC_threshold
df = df[df['log_FC'] > log_FC_threshold]
df = df.reset_index(drop=True)

#     return df

In [ ]:
cluster_markers = df
cluster_markers_list.append({'cluster id': cluster_idx, 'markers': cluster_markers})
cluster_markers_list

In [132]:
 OUTPUT_PATH = r'/storage/md_keren/shitay/outputs/clustering/markers/29.5.21'

writer_path = join(OUTPUT_PATH, 'markers_all_solutions.xlsx')
writer = pd.ExcelWriter(writer_path, engine='xlsxwriter')
for K_dir in sorted(os.listdir(OUTPUT_PATH)):
    working_dir = join(OUTPUT_PATH, K_dir)
    K = K_dir.split('_')[-1]
    print(K)

    new_table = {}
    for cluster_marker_list in sorted(os.listdir(working_dir)):
        cluster_idx = cluster_marker_list.split('_')[-1].replace('.csv', '')
        print(cluster_marker_list)
        df = pd.read_csv(join(working_dir, cluster_marker_list))
        new_table[int(cluster_idx)] = list(df['gene names'])[:100]
        if len(new_table[int(cluster_idx)]) < 100:
            new_table[int(cluster_idx)] = new_table[int(cluster_idx)] + [None] * (
                        100 - len(new_table[int(cluster_idx)]))
    print([len(vv) for vv in list(new_table.values())])
    new_sheet = pd.DataFrame(new_table)
    new_sheet = new_sheet.reindex(sorted(new_sheet.columns), axis=1)
    new_sheet.to_excel(writer, sheet_name=K, index=False)
writer.save()

10
markers_cluster_1.csv
markers_cluster_10.csv
markers_cluster_2.csv
markers_cluster_3.csv
markers_cluster_4.csv
markers_cluster_5.csv
markers_cluster_6.csv
markers_cluster_7.csv
markers_cluster_8.csv
markers_cluster_9.csv
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
11
markers_cluster_1.csv
markers_cluster_10.csv
markers_cluster_11.csv
markers_cluster_2.csv
markers_cluster_3.csv
markers_cluster_4.csv
markers_cluster_5.csv
markers_cluster_6.csv
markers_cluster_7.csv
markers_cluster_8.csv
markers_cluster_9.csv
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
12
markers_cluster_1.csv
markers_cluster_10.csv
markers_cluster_11.csv
markers_cluster_12.csv
markers_cluster_2.csv
markers_cluster_3.csv
markers_cluster_4.csv
markers_cluster_5.csv
markers_cluster_6.csv
markers_cluster_7.csv
markers_cluster_8.csv
markers_cluster_9.csv
[100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
13
markers_cluster_1.csv
markers_cluster_10.csv
markers_cluster_11.csv
markers_cluster_12.c